In [1]:
import sys
sys.path.append('/home/ckohler/workspace/bdai/projects/_experimental/fail/')

import torch
import numpy as np

import escnn
from escnn import nn as enn
from escnn import gspaces
from escnn import group

from fail.model.so2_transformer import SO2MultiheadAttention, SO2EncoderBlock, SO2TransformerEncoder, SO2Transformer
from fail.model.modules import SO2PoseForceEncoder

In [2]:
G = group.so2_group()
gspace = gspaces.no_base_space(G)

In [3]:
t = G.bl_regular_representation(L=5)
id = 11

In [4]:
model_dim = 8
hidden_dim = 8
out_dim = 1
in_type = enn.FieldType(gspace, [G.irrep(1) + G.irrep(0) + G.irrep(1) + G.irrep(0) + G.irrep(1) + G.irrep(0)])
num_heads = 4
# m = SO2Transformer(
#     in_type=in_type,
#     L=5,
#     model_dim=model_dim,
#     out_dim=out_dim,
#     num_heads=num_heads,
#     num_layers=4, 
#     dropout=0.0,
#     input_dropout=0.0
# )

In [18]:
m = SO2PoseForceEncoder(in_type, L=5, z_dim=8)

In [6]:
x = torch.randn(1, 20, 9)
print(x.shape)

torch.Size([1, 20, 9])


In [19]:
y = m(x)
y.shape

torch.Size([10, 88])

In [20]:
torch.linalg.norm(y.tensor, dim=1)

tensor([31.5756, 46.5735, 46.1434, 32.4250, 40.3306, 43.4274, 46.3095, 51.5190,
        44.5200, 49.5368], grad_fn=<LinalgVectorNormBackward0>)

In [21]:
np.set_printoptions(linewidth=10000, precision=4, suppress=True)

m.eval()
B = 10
L = 20
x = torch.randn(B, L, 9)

with torch.no_grad():
    y = m(x)
    print("Outputs' magnitudes")
    print(torch.linalg.norm(y.tensor, dim=1).numpy().reshape(-1)[:10])
    print('##########################################################################################')
    print("Errors' magnitudes")
    for r in range(8):
        # sample a random rotation
        g = G.sample()
        
        x_transformed = (g @ m.in_type(x.view(B*L, -1))).tensor.view(B,L,-1)
        x_transformed = x_transformed

        y_transformed = m(x_transformed)
        
        # verify that f(g@x) = g@f(x)=g@y
        print(torch.linalg.norm(y_transformed.tensor - (g @ y).tensor, dim=1).numpy().reshape(-1)[:10])
        

print('##########################################################################################')
print()

Outputs' magnitudes
[19.7772 15.1012 21.7488 20.5085 21.7323 24.6136 17.2663 17.8747 22.602  18.5206]
##########################################################################################
Errors' magnitudes
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
##########################################################################################



In [10]:
in_type

[SO(2): {[[[[[irrep_1]_[irrep_0]]_[irrep_1]]_[irrep_0]]_[irrep_1]]_[irrep_0] (x1)}(9)]

In [11]:
t

SO(2)|[regular_5]:11

In [12]:
gspace.type(G.standard_representation())

[SO(2): {irrep_1 (x1)}(2)]

In [13]:
gspace.type(G.irrep(1))

[SO(2): {irrep_1 (x1)}(2)]

In [23]:
G.bl_regular_representation(L=7).size

15

In [15]:
y.type

[SO(2): {regular_5 (x1)}(11)]

In [16]:
y = y.tensor.view(1,-1)

In [17]:
y.shape

torch.Size([1, 110])